In [1]:
import json

In [2]:
import json
ENTITIES_FILEPATH='camera_trainingset/camera-ENTITIES.json'
JSON_FILES_FILEPATH = 'camera_trainingset/camera-JSON_FILES.json'
with open(ENTITIES_FILEPATH) as json_file:
    matching = json.load(json_file)
with open(JSON_FILES_FILEPATH) as json_file:
    json_filepaths = json.load(json_file)

In [3]:
len(json_filepaths)

29787

In [4]:
import os

filepaths = {}
DATASET_FILEPATH = 'camera_dataset/2013_camera_dataset/'
for filepath in json_filepaths:
    json_id = filepath['resource_id']
    domain = filepath['source_name']
    filename = str(filepath['json_number'])+'.json'
    filepaths[json_id]= os.path.join(DATASET_FILEPATH,domain,filename)

In [5]:
# positives samples è una tupla record1,record2,matching/nomatching
positives_samples = []
for entity in matching:
    instances = entity['instances']
    i=0
    for instance in instances:
        filepath = filepaths[instance]
        with open(filepath) as json_file:
            record = json.load(json_file)
            left_page_title = record['<page title>']
        for j in range(i+1,len(instances)):
            filepath = filepaths[instances[j]]
            with open(filepath) as json_file:
                record = json.load(json_file)
                right_page_title = record['<page title>']
                positives_samples.append((left_page_title,right_page_title,1))
        i += 1

In [6]:
positives_samples[200][2]

1

In [25]:
len(positives_samples)

244

In [7]:
all_sources = os.listdir('camera_dataset/2013_camera_dataset')

In [8]:
import random
random.choice(all_sources)

'cammarkt.com'

In [9]:
import numpy as np
GLOVE_DIR='glove'
GLOVE_FILENAME='glove.840B.300d.txt'
wordToEmbeddingMap = {}
with open(GLOVE_DIR+'/'+GLOVE_FILENAME) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, 'f', sep=' ')
        if len(coefs) != 0:
            wordToEmbeddingMap[word] = coefs

In [10]:
import nltk
stopwords = nltk.download("stopwords")

from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
tokenizer = RegexpTokenizer(r'\w+')
stop_words = set(stopwords.words('english'))
stop_words.update(['.', ',', '"', "'", ':', ';', '(', ')', '[', ']', '{', '}','|','-'])

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jgmathew/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [80]:
BASE_DIR ='camera_dataset/2013_camera_dataset/'
source1 = 'buy.net'
source2 = 'www.henrys.com'
source3 = 'www.shopbot.com.au'
source4 = 'www.canon-europe.com'

'''
while(source2==source1):
    source2 = random.choice(all_sources)
'''

all_files_source1 = os.listdir(BASE_DIR+source1)
all_files_source2 = os.listdir(BASE_DIR+source2)
all_files_source3 = os.listdir(BASE_DIR+source3)
all_files_source4 = os.listdir(BASE_DIR+source4)

In [81]:
all_json_source1 = list(filter(lambda s: s.endswith('.json'), all_files_source1))
all_json_source2 = list(filter(lambda s: s.endswith('.json'),all_files_source2))
all_json_source3 = list(filter(lambda s: s.endswith('.json'),all_files_source3))
all_json_source4 = list(filter(lambda s: s.endswith('.json'),all_files_source4))

In [82]:
all_json_source1 = random.sample(all_json_source1, 50)
all_json_source2 = random.sample(all_json_source2, 50)
all_json_source3 = random.sample(all_json_source3, 50)
all_json_source4 = random.sample(all_json_source4, 50)

In [83]:
print(source1)
print(len(all_files_source1))
print(len(all_json_source1))

buy.net
716
50


In [84]:
print(source2)
print(len(all_files_source2))
print(len(all_json_source2))

www.henrys.com
362
50


In [85]:
print(source3)
print(len(all_files_source3))
print(len(all_json_source3))

www.shopbot.com.au
1032
50


In [86]:
print(source4)
print(len(all_files_source4))
print(len(all_json_source4))

www.canon-europe.com
328
50


In [87]:
def extract_titles(filenamelist,source,basedir=BASE_DIR):
    page_titles = []
    for filename in filenamelist:
        filepath = os.path.join(BASE_DIR,source,filename)
        with open(filepath) as json_file:
            json_ob = json.load(json_file)
            page_title = json_ob['<page title>']
            page_titles.append(page_title)
    return page_titles                

In [88]:
def createDistributedRapresentation(recordTitleList):
    embeddingMap = {}
    for recordtitle in recordTitleList:
        currentTitle = recordtitle
        tokens_title = tokenizer.tokenize(currentTitle)
        filtered_title = [word for word in tokens_title if word not in stop_words]
        #processo di creazione del vettore di embedding
        numeratoreVec = np.zeros(300)
        ntokens = 0
        for token in filtered_title:
            ntokens += 1
            embeddingVector = wordToEmbeddingMap.get(token)
            if embeddingVector is not None:
                numeratoreVec += embeddingVector
        tupleEmbedding = numeratoreVec /ntokens
        embeddingMap[currentTitle]= tupleEmbedding
    return embeddingMap

In [89]:
titlesList_source1 = extract_titles(all_json_source1,source1)
titlesList_source2 = extract_titles(all_json_source2,source2)
titlesList_source3 = extract_titles(all_json_source3,source3)
titlesList_source4 = extract_titles(all_json_source4,source4)

In [90]:
embedding_source1 = createDistributedRapresentation(titlesList_source1)
embedding_source2 = createDistributedRapresentation(titlesList_source2)
embedding_source3 = createDistributedRapresentation(titlesList_source3)
embedding_source4 = createDistributedRapresentation(titlesList_source4)

In [91]:
from sklearn.metrics.pairwise import cosine_similarity

In [92]:
print(len(embedding_source1))
print(len(titlesList_source1))

50
50


In [93]:
print(len(embedding_source2))
print(len(titlesList_source2))

50
50


In [94]:
print(len(embedding_source3))
print(len(titlesList_source3))

50
50


In [95]:
print(len(embedding_source4))
print(len(titlesList_source4))

50
50


In [96]:
'''
for i in range(len(titlesList_source2)):
    for j in range(i+1, len(titlesList_source2)):
        if titlesList_source2[i] == titlesList_source2[j]:
            print('found duplicate records')
            print(i)
            print(j)
            print(titlesList_source2[i])
            print()
            print()
'''

"\nfor i in range(len(titlesList_source2)):\n    for j in range(i+1, len(titlesList_source2)):\n        if titlesList_source2[i] == titlesList_source2[j]:\n            print('found duplicate records')\n            print(i)\n            print(j)\n            print(titlesList_source2[i])\n            print()\n            print()\n"

In [99]:
import itertools

titlePairs1 = list(itertools.product(titlesList_source1, titlesList_source2))
titlePairs2 = list(itertools.product(titlesList_source3, titlesList_source4))


print(len(titlePairs1))
print(len(titlePairs2))

2500
2500


In [107]:
print(type(titlePairs[0][1]))

<class 'str'>


In [109]:
titlePairs[45]

('Canon EOS 70D Black SLR Digital Camera Body Only (20.2 MP, SD Card Slot) Price Comparison at Buy.net',
 'SAMSUNG WB350F RED 16MP 21X 3" SMART CAMERA EC-WB350FBPRCA')

In [114]:
THRESHOLD = 0.9

def isMatching(firstTitle, secondTitle, threshold, firstDict, secondDict):
    firstEmbedding = firstDict[firstTitle].reshape(1,-1)
    secondEmbedding = secondDict[secondTitle].reshape(1,-1)
    similarityScore = cosine_similarity(firstEmbedding,secondEmbedding)[0][0]
    return similarityScore > threshold

In [115]:
filteredTitlePairs1 = list(filter(lambda x: not isMatching(x[0],x[1],THRESHOLD, embedding_source1, embedding_source2), titlePairs1))
filteredTitlePairs2 = list(filter(lambda x: not isMatching(x[0],x[1],THRESHOLD, embedding_source3, embedding_source4), titlePairs2))

In [116]:
print(len(filteredTitlePairs1))
print(len(filteredTitlePairs2))

2500
2500


In [118]:
filteredTitlePairs = filteredTitlePairs1 + filteredTitlePairs2
print(len(filteredTitlePairs))
print(type(filteredTitlePairs[0]))
print(type(filteredTitlePairs[0][1]))

5000
<class 'tuple'>
<class 'str'>


In [121]:
firstTestTitle, secondTestTitle = filteredTitlePairs[0]
print(firstTestTitle)
print()
print(secondTestTitle)

Canon EOS 70D Black SLR Digital Camera Body Only (20.2 MP, SD Card Slot) Price Comparison at Buy.net

CANON POWERSHOT D30 12.1MP 5X 3" WATERPROOF 9337B001


In [122]:
csvData = []

# append matching records
for matchingRow in positives_samples:
    row = [matchingRow[0], matchingRow[1], matchingRow[2]]
    csvData.append(row)

print(len(csvData))
print()
print(csvData[0])

244

['Nikon Coolpix S6800 Digital Camera (Black) | UK Digital Cameras', 'NIKON COOLPIX S6800 RED 16MP 12X 32182', 1]


In [123]:
# append non-matching rows
for nonMatchingRow in filteredTitlePairs:
    row = [nonMatchingRow[0], nonMatchingRow[1], 0]
    csvData.append(row)
    
print(len(csvData))
print()
print(csvData[2243])

5244

['VTech Kidizoom 80-077341 2 Megapixel Compact Camera (1.8" LCD - 1600 X 1200 Image - 320 X 240 Video) Price Comparison at Buy.net', 'SONY DSCH400 BLACK 20.1MP 63X 3" DSCH400B', 0]


In [124]:
random.shuffle(csvData)

In [143]:
from sklearn.model_selection import train_test_split

trainCsvData, testCsvData = train_test_split(csvData, test_size=0.20, random_state=42)

In [144]:
print(len(trainCsvData))
print(len(testCsvData))

4195
1049


In [145]:
testCsvData, valCsvData = train_test_split(testCsvData, test_size=0.50, random_state=42)

In [146]:
print(len(testCsvData))
print(len(valCsvData))

524
525


In [150]:
def getNoOfMatchingRecords(data):
    counter = 0
    for row in data:
        label = row[2]
        if label == 1:
            counter += 1
    return counter

In [152]:
print(getNoOfMatchingRecords(trainCsvData))
print(getNoOfMatchingRecords(testCsvData))
print(getNoOfMatchingRecords(valCsvData))

193
23
28


In [153]:
import csv 

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

def writeDataset(data, filename):
    with open(filename, 'w') as f:
        writer = csv.writer(f, dialect='myDialect')
        writer.writerow(['attributi_x', 'attributi_y', 'label'])
        for row in data:
            writer.writerow(row)

In [155]:
writeDataset(trainCsvData, 'datasets/DI2KG/DI2KG_train.csv')
writeDataset(testCsvData, 'datasets/DI2KG/DI2KG_test.csv')
writeDataset(valCsvData, 'datasets/DI2KG/DI2KG_val.csv')